#Import

In [1]:
#!pip install numpy==2.0.2
#!pip install tensorflow==2.18.0
#!pip install transformers==4.51.3
#!pip install torch==2.6.0
#!pip install keras==3.7.0

In [2]:
import pandas as pd
import numpy as np

from transformers import AutoTokenizer
from transformers import TFDistilBertForSequenceClassification

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [3]:
# Intentions file import
intentions = pd.read_csv("/content/test_intent_dataset.csv")

#Model

In [4]:
# Convert intention labels into numerical form
label_encoder = LabelEncoder()
intentions['encoded_labels'] = label_encoder.fit_transform(intentions['intent'])

# Split the data
X_train, X_test, Y_train, Y_test = train_test_split(intentions["text"].tolist(), intentions["encoded_labels"].tolist(), test_size=0.2, random_state=42)
Y_train, Y_test = np.array(Y_train), np.array(Y_test)

# Tokenization
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

train_tokens = tokenizer(X_train, padding=True, truncation=True, max_length=64, return_tensors="tf")
test_tokens = tokenizer(X_test, padding=True, truncation=True, max_length=64, return_tensors="tf")

# Shuffle and batch data for input
train_data = tf.data.Dataset.from_tensor_slices((dict(train_tokens), Y_train)).shuffle(1000).batch(8)
test_data = tf.data.Dataset.from_tensor_slices((dict(test_tokens), Y_test)).batch(8)

# Model initialization
model = TFDistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-multilingual-cased", num_labels=len(label_encoder.classes_)
)

# Model compilation
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTr

In [5]:
# Train model
model.fit(
    train_data,
    validation_data=test_data,
    epochs=10
)

Epoch 1/10
8/8 [==============================] - 67s 5s/step - loss: 2.4659 - accuracy: 0.1404 - val_loss: 2.4389 - val_accuracy: 0.0000e+00
Epoch 2/10
8/8 [==============================] - 27s 3s/step - loss: 2.2908 - accuracy: 0.2632 - val_loss: 2.3687 - val_accuracy: 0.0667
Epoch 3/10
8/8 [==============================] - 27s 3s/step - loss: 2.0491 - accuracy: 0.4912 - val_loss: 2.2110 - val_accuracy: 0.2000
Epoch 4/10
8/8 [==============================] - 27s 3s/step - loss: 1.8062 - accuracy: 0.6491 - val_loss: 2.0394 - val_accuracy: 0.2667
Epoch 5/10
8/8 [==============================] - 26s 3s/step - loss: 1.5129 - accuracy: 0.7544 - val_loss: 1.8981 - val_accuracy: 0.4667
Epoch 6/10
8/8 [==============================] - 26s 3s/step - loss: 1.2010 - accuracy: 0.8772 - val_loss: 1.7618 - val_accuracy: 0.5333
Epoch 7/10
8/8 [==============================] - 32s 3s/step - loss: 0.9392 - accuracy: 0.9298 - val_loss: 1.6002 - val_accuracy: 0.6000
Epoch 8/10
8/8 [==============

In [ ]:
# Save Model
#model.save_pretrained("./intent_model")
#tokenizer.save_pretrained("./intent_model")

# Load Model
#model = TFDistilBertForSequenceClassification.from_pretrained("./intent_model")
#tokenizer = AutoTokenizer.from_pretrained("./intent_model")

In [13]:
sample_texts = ["Hello. Can you help me?", "Bye"]
sample_tokens = tokenizer(sample_texts, padding=True, truncation=True, max_length=64, return_tensors="tf")

predictions = model.predict(dict(sample_tokens)).logits
predicted_labels = np.argmax(predictions, axis=1)

# Decode the predicted class numbers back to label names
decoded_predictions = label_encoder.inverse_transform(predicted_labels)
print(decoded_predictions)

1/1 [==============================] - 0s 149ms/step
['help_request' 'goodbye']


In [16]:
def respond_to_input(text):
    tokens = tokenizer([text], padding=True, truncation=True, max_length=64, return_tensors="tf")
    logits = model.predict(dict(tokens)).logits
    pred = np.argmax(logits, axis=1)
    intent = label_encoder.inverse_transform(pred)[0]

    response = intent_response_map.get(intent, "Sorry, I didn't quite understand that.")
    return intent, response

# Try it out
input_text = "I want to cancel my booking"
intent, response = respond_to_input(input_text)
print(f"Intent: {intent}")
print(f"Response: {response}")


1/1 [==============================] - 0s 82ms/step
Intent: payment
Response: Sorry, I didn't quite understand that.


In [18]:
intentions["intent"].unique()

array(['greeting', 'goodbye', 'thanks', 'help_request', 'report_issue',
       'book_flight', 'book_hotel', 'book_taxi', 'book_table',
       'order_status', 'payment', 'product_info'], dtype=object)